# Deep Dive into LangChain
## LLMs, Prompt Templates, Caching, Streaming, Chains

This notebook uses the latest versions of the OpenAI and LangChain libraries.

In [ ]:
pip install -r ./requirements.txt -q

Download [requirements.txt](https://drive.google.com/file/d/1UpURYL9kqjXfe9J8o-_Dq5KJTbQpzMef/view?usp=sharing)

In [3]:
pip install --upgrade -q langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 9.2 MB/s eta 0:00:00


In [4]:
pip install --upgrade -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00


In [ ]:
# pip show openai

In [2]:
pip show langchain

In [7]:
pip install langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.1 MB/s eta 0:00:00


### Python-dotenv

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

# loading the API Keys from .env
load_dotenv(find_dotenv(), override=True)

# os.environ.get('OPENAI_API_KEY')

True

## Chat Models: GPT-3.5 Turbo and GPT-4

In [9]:
from langchain_openai import ChatOpenAI
from google.colab import userdata

llm = ChatOpenAI(openai_api_key=userdata.get('OPENAI_API_KEY'))

# invoking the llm (running the prompt)
output = llm.invoke('Explain quantum mechanics in one sentence.', model='gpt-3.5-turbo', temperature=0.1)
print(output.content)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
# help(ChatOpenAI)  # see the llm constructor arguments with its defaults

In [ ]:
# using Chat Completions API Messages: System, Assistant and Human
from langchain.schema import(
    SystemMessage,
    AIMessage,
    HumanMessage
)
messages = [
    SystemMessage(content='You are a physicist and respond only in German.'),
    HumanMessage(content='Explain quantum mechanics in one sentence.')
]

output = llm.invoke(messages)
print(output.content)

Quantenmechanik beschreibt das Verhalten von Teilchen auf subatomarer Ebene, indem sie sowohl Wellen- als auch Teilcheneigenschaften besitzen.


## Caching LLM Responses

### 1. In-Memory Cache

In [ ]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI
llm = OpenAI(model_name='gpt-3.5-turbo-instruct')

In [ ]:
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())

In [ ]:
%%time
prompt = 'Tell a me a joke that a toddler can understand.'
llm.invoke(prompt)

CPU times: user 7.42 ms, sys: 0 ns, total: 7.42 ms
Wall time: 700 ms


"\n\nWhy did the banana go to the doctor?\n\nBecause it wasn't peeling well! "

In [ ]:
%%time
llm.invoke(prompt)

CPU times: user 120 µs, sys: 110 µs, total: 230 µs
Wall time: 233 µs


"\n\nWhy did the banana go to the doctor?\n\nBecause it wasn't peeling well! "

### 2. SQLite Caching

In [ ]:
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [ ]:
%%time
# First request (not in cache, takes longer)
llm.invoke("Tell me a joke")

CPU times: user 18.1 ms, sys: 0 ns, total: 18.1 ms
Wall time: 833 ms


'\n\nWhy was the math book sad?\nBecause it had too many problems.'

In [ ]:
%%time
# Second request (cached, faster)
llm.invoke("Tell me a joke")

CPU times: user 35.9 ms, sys: 39.4 ms, total: 75.3 ms
Wall time: 74.4 ms


'\n\nWhy was the math book sad?\nBecause it had too many problems.'

## LLM Streaming

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
prompt = 'Write a rock song about the Moon and a Raven.'
print(llm.invoke(prompt).content)

(Verse 1)
Under the silver moonlight, the raven takes flight,
Across the midnight sky, with wings so dark and wide.
A mystical creature, a messenger of the night,
Guiding me through the shadows, with its piercing eyes.

(Pre-Chorus)
Oh moon, shining bright, your glow illuminates the night,
And the raven, in its grace, dances with the starry haze.
They tell tales of secrets, hidden in the lunar phase,
In this rock anthem, their legend we embrace.

(Chorus)
Moon and Raven, so wild and untamed,
A cosmic alliance, forever unchained.
Their spirits entwined, in the darkest domain,
This rock song echoes their eternal refrain.

(Verse 2)
The moon, a celestial queen, reflecting ancient dreams,
A beacon in the darkness, igniting hope it seems.
The raven, a rebel, defying rules and norms,
Together they transcend, beyond earthly forms.

(Bridge)
In the lunar abyss, they find solace and bliss,
An unbreakable bond, a tale seldom amiss.
They navigate the heavens, defying gravity's call,
Through moonl

In [ ]:
for chunk in llm.stream(prompt):
    print(chunk.content, end='', flush=True)

(Verse 1)
In the dead of night, beneath the silver light,
The moon's glow illuminates the sky so bright,
A raven takes flight, with feathers black as coal,
Its piercing eyes, a story yet untold.

(Pre-Chorus)
Through the shadows, they dance, their destinies entwined,
The moon and the raven, an alliance undefined,
Together they soar, in the depths of the night,
A tale of darkness and pure lunar might.

(Chorus)
Moon and raven, a celestial duet,
Unleashing power, in every silhouette,
They rule the night, a symphony untamed,
Through the darkness, their legacy proclaimed.

(Verse 2)
On lunar beams, the raven finds its way,
Across the night sky, where secrets gently sway,
The moon whispers secrets with a tranquil glow,
As the raven listens, their bond continues to grow.

(Bridge)
In the moonlit haze, they share a cosmic dance,
Their spirits intertwining in a lunar trance,
Together they conquer, the night's eternal reign,
Their union unbreakable, forever to remain.

(Chorus)
Moon and raven, 

## PromptTemplates

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Define a template for the prompt
template = '''You are an experienced virologist.
Write a few sentences about the following virus "{virus}" in {language}.'''

# Create a PromptTemplate object from the template
prompt_template = PromptTemplate.from_template(template=template)

# Fill in the variable: virus and language
prompt = prompt_template.format(virus='hiv', language='german')
prompt  # Returns the generated prompt


'You are an experienced virologist.\nWrite a few sentences about the following virus "hiv" in german.'

In [ ]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
output = llm.invoke(prompt)
print(output.content)

HIV, das humane Immundefizienz-Virus, ist ein Retrovirus, das das menschliche Immunsystem angreift. Es wird hauptsächlich durch ungeschützten Geschlechtsverkehr, den Austausch von infizierten Nadeln oder von Mutter zu Kind während der Schwangerschaft, Geburt oder Stillzeit übertragen. HIV kann zu AIDS führen, einer schweren Erkrankung, bei der das Immunsystem geschwächt ist und der Körper anfällig für verschiedene Infektionen und Krankheiten wird. Obwohl es keine Heilung für HIV gibt, kann die rechtzeitige Diagnose und Behandlung das Fortschreiten der Krankheit verlangsamen und die Lebensqualität der Betroffenen verbessern.


## ChatPromptTemplates

In [ ]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

# Create a chat template with system and human messages
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='You respond only in the JSON format.'),
        HumanMessagePromptTemplate.from_template('Top {n} countries in {area} by population.')
    ]
)

# Fill in the specific values for n and area
messages = chat_template.format_messages(n='5', area='World')
print(messages)  # Outputs the formatted chat messages


[SystemMessage(content='You respond only in the JSON format.'), HumanMessage(content='Top 5 countries in World by population.')]


In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()
output = llm.invoke(messages)
print(output.content)

{
  "data": [
    {
      "country": "China",
      "population": 1444216107
    },
    {
      "country": "India",
      "population": 1393409038
    },
    {
      "country": "United States",
      "population": 332915073
    },
    {
      "country": "Indonesia",
      "population": 276361783
    },
    {
      "country": "Pakistan",
      "population": 225199937
    }
  ]
}


## Simple Chains

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI()
template = '''You are an experience virologist.
Write a few sentences about the following virus "{virus}" in {language}.'''
prompt_template = PromptTemplate.from_template(template=template)

chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

output = chain.invoke({'virus': 'HSV', 'language': 'Spanish'})




> Entering new LLMChain chain...
Prompt after formatting:
You are an experience virologist.
Write a few sentences about the following virus "HSV" in Spanish.

> Finished chain.


In [ ]:
print(output)

{'virus': 'HSV', 'language': 'Spanish', 'text': 'El virus del herpes simple (HSV) es un virus altamente contagioso que afecta a los seres humanos. Se divide en dos tipos: HSV-1 y HSV-2. El HSV-1 generalmente se asocia con infecciones orales, como el herpes labial, mientras que el HSV-2 está relacionado con infecciones genitales. Ambos tipos de HSV pueden causar brotes recurrentes a lo largo de la vida de una persona, aunque los síntomas pueden variar desde leves hasta graves. Actualmente no existe una cura para el HSV, pero hay medicamentos disponibles que pueden ayudar a controlar los síntomas y reducir la frecuencia de los brotes.'}


In [ ]:
template = 'What is the capital of {country}?. List the top 3 places to visit in that city. Use bullet points'
prompt_template = PromptTemplate.from_template(template=template)

# Initialize an LLMChain with the ChatOpenAI model and the prompt template
chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

country = input('Enter Country: ')

# Invoke the chain with specific virus and language values
output = chain.invoke(country)
print(output['text'])

Enter Country:  Spain




> Entering new LLMChain chain...
Prompt after formatting:
What is the capital of Spain?. List the top 3 places to visit in that city. Use bullet points

> Finished chain.
The capital of Spain is Madrid. Here are the top 3 places to visit in Madrid:

- The Royal Palace: Explore this magnificent 18th-century palace, which is the official residence of the Spanish Royal Family. Admire the stunning architecture, luxurious rooms, and beautiful gardens.
- Prado Museum: Immerse yourself in art at the Prado Museum, one of the world's finest art galleries. Discover masterpieces by renowned Spanish artists such as Velázquez, Goya, and El Greco.
- Retiro Park: Escape the hustle and bustle of the city and relax in Retiro Park. This vast green oasis offers peaceful gardens, a boating lake, fountains, and even a beautiful glass palace.


## Sequential Chains

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

# Initialize the first ChatOpenAI model (gpt-3.5-turbo) with specific temperature
llm1 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)

# Define the first prompt template
prompt_template1 = PromptTemplate.from_template(
    template='You are an experienced scientist and Python programmer. Write a function that implements the concept of {concept}.'
)
# Create an LLMChain using the first model and the prompt template
chain1 = LLMChain(llm=llm1, prompt=prompt_template1)

# Initialize the second ChatOpenAI model (gpt-4-turbo) with specific temperature
llm2 = ChatOpenAI(model_name='gpt-4-turbo-preview', temperature=1.2)

# Define the second prompt template
prompt_template2 = PromptTemplate.from_template(
    template='Given the Python function {function}, describe it as detailed as possible.'
)
# Create another LLMChain using the second model and the prompt template
chain2 = LLMChain(llm=llm2, prompt=prompt_template2)

# Combine both chains into a SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

# Invoke the overall chain with the concept "linear regression"
output = overall_chain.invoke('linear regression')




> Entering new SimpleSequentialChain chain...
Sure! Here's an example of a function that implements the concept of linear regression in Python:

```python
import numpy as np

def linear_regression(x, y):
    # Calculate the number of data points
    n = len(x)
    
    # Calculate the mean of x and y
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    
    # Calculate the slope (beta1) and intercept (beta0) of the regression line
    numerator = np.sum((x - x_mean) * (y - y_mean))
    denominator = np.sum((x - x_mean) ** 2)
    beta1 = numerator / denominator
    beta0 = y_mean - beta1 * x_mean
    
    # Return the slope and intercept of the regression line
    return beta1, beta0
```

This function takes two arrays, `x` and `y`, as inputs. `x` represents the independent variable and `y` represents the dependent variable. The function calculates the slope (`beta1`) and intercept (`beta0`) of the regression line using the least squares method. It then returns these values.

To use th

In [ ]:
print(output['output'])

The Python function `linear_regression` is an implementation of simple linear regression, which is a fundamental statistical approach used to model the relationship between a single independent variable (denoted as `x`) and a dependent variable (denoted as `y`). The main goal of this function is to find the best-fitting straight line (also known as the regression line) that describes this relationship. The equation of this line is `y = beta1 * x + beta0`, where `beta1` is the slope of the line, and `beta0` is the y-intercept.

The implementation details of the `linear_regression` function are explained below:

1. **Imports and Dependencies**: The function uses NumPy (imported as `np`), a popular library for numerical computing in Python. NumPy provides efficient ways to handle arrays and mathematical operations, making it an ideal choice for implementing linear regression.

2. **Parameters**: The function accepts two parameters, `x` and `y`, which are NumPy arrays of the same length. T

In [ ]:
pip install -q langchain_experimental

In [ ]:
from langchain_experimental.utilities import PythonREPL
python_repl = PythonREPL()
python_repl.run('print([n for n in range(1, 100) if n % 13 == 0])')

Python REPL can execute arbitrary code. Use with caution.


'[13, 26, 39, 52, 65, 78, 91]\n'

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

# Initialize the ChatOpenAI model with gpt-4-turbo and a temperature of 0
llm = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0)

# Create a Python agent using the ChatOpenAI model and a PythonREPLTool
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)

# Invoke the agent
prompt = 'Calculate the square root of the factorial of 12 and display it with 4 decimal points'
agent_executor.invoke(prompt)



> Entering new AgentExecutor chain...
To solve this, I need to calculate the factorial of 12 first, then find its square root, and finally format the result to display it with 4 decimal points. I can use the `math` module in Python for both the factorial and square root calculations.
Action: Python_REPL
Action Input: import math
print(f"{math.sqrt(math.factorial(12)):.4f}")
Observation: 21886.1052

Thought:I now know the final answer
Final Answer: 21886.1052

> Finished chain.


{'input': 'Calculate the square root of the factorial of 12 and display it with 4 decimal points',
 'output': '21886.1052'}

In [ ]:
response = agent_executor.invoke('What is the answer to 5.1 ** 7.3?')



> Entering new AgentExecutor chain...
I need to calculate 5.1 raised to the power of 7.3 to get the answer.
Action: Python_REPL
Action Input: print(5.1 ** 7.3)
Observation: 146306.05007233328

Thought:I now know the final answer
Final Answer: 146306.05007233328

> Finished chain.


In [ ]:
response

{'input': 'What is the answer to 5.1 ** 7.3?', 'output': '146306.05007233328'}

In [ ]:
print(response['input'])

What is the answer to 5.1 ** 7.3?


In [ ]:
print(response['output'])


146306.05007233328
